<a href="https://colab.research.google.com/github/mshsu/stat400-sets/blob/main/stock_scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup

In [26]:
# Instantiate our list of rows
rows = []

# Business Insider has sites for each of these indices, so we'll query each one.
# In this for-loop, each stock index is represented by a tuple. 
#   [0] is the string literal in the link
#   [1] is the index name
#   [2] is the number of pages the site takes to display the entire table.
# We iterate over each one.
for idx in [('dow_jones', 'Dow Jones', 1), 
            ('s&p_500', 'S&P 500', 10), 
            ('nasdaq_100', 'Nasdaq 100', 2)]:
  # Here we iterate over each page
  for i in range(idx[2]):
    # Query the link:
    link = "https://markets.businessinsider.com/index/components/" + idx[0]
    if i > 0: link = link + "?p=" + str(i+1)
    response = requests.get(link)
    soup = BeautifulSoup(response.content, "html.parser")

    # The data are stored in an HTML element called "tbody"
    table = soup.find("tbody", {"class": "table__tbody"})

    # Each row is represented by the "tr" HTML element, so we iterate over each
    for tr in table.find_all("tr"):
      # Each value in the row is represented by the "td" element
      info = tr.find_all("td")
      # Here we construct the row in dictionary format
      # We also clean the data a little in the process
      rows.append({
          "Name": info[0].text.strip("\n"),
          "Index": idx[1],
          "Latest Price": info[1].text.strip("\t").split("\n")[1],
          "Previous Close": info[1].text.strip("\t").split("\n")[2],
          "Low": info[2].text.strip("\t").split("\n")[1],
          "High": info[2].text.strip("\t").split("\n")[2],
          "Change": info[3].text.split("\n")[1],
          "% Change": info[3].text.split("\n")[2],
          "Datetime": info[4].text.split("\n")[1],
          "3 Mo. Change": info[5].text.split("\n")[1],
          "3 Mo. % Change": info[5].text.split("\n")[2],
          "6 Mo. Change": info[6].text.split("\n")[1],
          "6 Mo. % Change": info[6].text.split("\n")[2],
          "1 Year Change": info[7].text.split("\n")[1],
          "1 Year % Change": info[7].text.split("\n")[2]
          })
    
    # We then delay our next loop so we don't get blocked...
    time.sleep(1)  

# Turn rows into tabular dataframe
Stocks = pd.DataFrame(rows)
# Export to CSV file
Stocks.to_csv('Stocks.csv', index=False)
# Print here
Stocks

,Name,Index,Latest Price,Previous Close,Low,High,Change,% Change,Datetime,3 Mo. Change,3 Mo. % Change,6 Mo. Change,6 Mo. % Change,1 Year Change,1 Year % Change
0,3M,Dow Jones,106.06,105.76,105.95,107.10,0.30,0.28%,01:10 PM04/17/2023 01:10:32 PM UTC-0400,-25.15,-19.42%,-9.86,-8.63%,-44.29,-29.79%
1,American Express,Dow Jones,162.15,163.24,160.89,162.89,-1.09,-0.67%,01:10 PM04/17/2023 01:10:41 PM UTC-0400,3.56,2.29%,17.73,12.52%,-20.39,-11.35%
2,Amgen,Dow Jones,248.14,250.00,248.06,251.50,-1.87,-0.75%,01:09 PM04/17/2023 01:09:54 PM UTC-0400,-22.23,-8.18%,-2.16,-0.86%,-4.01,-1.58%
3,Apple,Dow Jones,164.45,165.21,164.03,165.39,-0.76,-0.46%,01:10 PM04/17/2023 01:10:40 PM UTC-0400,25.34,18.80%,17.11,11.97%,-10.30,-6.04%
4,Boeing,Dow Jones,203.76,201.66,200.84,206.10,2.10,1.04%,01:10 PM04/17/2023 01:10:40 PM UTC-0400,-1.84,-0.86%,79.88,60.33%,29.32,16.03%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592,Walgreens Boots Alliance,Nasdaq 100,35.59,35.32,35.35,35.74,0.27,0.76%,12:21 PM04/17/2023 12:21:25 PM UTC-0400,-1.09,-2.96%,2.04,6.06%,-8.91,-19.98%
593,Warner Bros. Discovery,Nasdaq 100,14.18,13.78,13.76,14.22,0.40,2.90%,12:21 PM04/17/2023 12:21:35 PM UTC-0400,0.92,7.00%,1.56,12.48%,-11.94,-45.92%
594,Workda a,Nasdaq 100,194.87,194.04,193.29,195.38,0.83,0.43%,12:21 PM04/17/2023 12:21:03 PM UTC-0400,24.59,14.71%,49.03,34.36%,-36.92,-16.15%
595,Xcel Energy,Nasdaq 100,70.19,70.14,70.11,70.86,0.05,0.07%,12:21 PM04/17/2023 12:21:19 PM UTC-0400,-0.13,-0.18%,11.30,18.92%,-3.33,-4.48%
